<a href="https://colab.research.google.com/github/Doubles2/All_of_DoubleS2/blob/master/AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

data load

In [2]:
# 변수명은 X_Train, Y_Train, X_Test, Y_Test

# # 아래 코드는 선택해서 업로드하는 방법
# from google.colab import files
# uploaded = files.upload()

# for fn in iploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

#구글 드라이브에서 업로드
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
X_Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/train_data.csv')
Y_Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/train_label.csv')
X_Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/test_data.csv')
Y_Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/test_label.csv')

In [4]:
print('X_Train : ', X_Train.shape)
print('Y_Train : ', Y_Train.shape)
print('X_Test : ', X_Test.shape)
print('Y_Test : ', Y_Test.shape)

X_Train :  (201686, 22)
Y_Train :  (201686, 1)
X_Test :  (50422, 22)
Y_Test :  (50422, 1)


In [5]:
X_Train.head()

,visit_dow,visit_year,visit_month,holiday_flg,min_visitors,max_visitors,mean_visitors,median_visitors,count_visitors,air_genre_name,air_area_name,latitude,longitude,rv_sum,rv_mean,rvd_mean,date_int,var_max_lat,var_max_long,long_plus_lat,air_store_id_label,weight
0,5,2016,10,0,3.0,20.0,9.243902,9.0,41.0,8.0,70.0,35.646572,139.653247,13.751283,5.084251,5.500971,20161008,8.374060,4.620151,175.299819,386,0.048283
1,1,2016,10,0,1.0,45.0,16.871795,13.0,39.0,7.0,26.0,34.756950,134.841177,13.751283,5.084251,5.500971,20161004,9.263682,9.432221,169.598128,88,0.044954
2,0,2016,5,0,2.0,17.0,8.015152,7.5,66.0,11.0,79.0,35.699566,139.636438,13.751283,5.084251,5.500971,20160530,8.321066,4.636960,175.336004,376,0.002125
3,2,2016,11,0,2.0,65.0,19.984848,17.0,66.0,4.0,0.0,33.589216,130.392813,25.000000,12.500000,8.500000,20161109,10.431416,13.880585,163.982029,822,0.082641
4,2,2016,7,0,7.0,47.0,26.048780,25.0,41.0,11.0,96.0,34.687697,135.495413,13.751283,5.084251,5.500971,20160706,9.332935,8.777985,170.183110,109,0.006358


정규화

In [0]:
def min_max_norm(mmnTable):
  import math
  import numpy as np

  alp = math.pow(10, -8)
  mmnTable_col = mmnTable.columns
  for col in mmnTable_col:
    mmnTable[col] = ( mmnTable[col] - mmnTable[col].min() ) / ( mmnTable[col].max() - mmnTable[col].min() + alp)
  return mmnTable

def z_norm(zTable):
  import math
  import numpy as np

  alp = math.pow(10, -8)
  zTable_col = zTable.columns
  for col in zTable_col:
    zTable[col] = ( zTable[col] - zTable[col].mean() ) / zTable[col].std()
  return zTable

In [0]:
#X_Train = z_norm(X_Train)
#X_Test = z_norm(X_Test)
X_Train = min_max_norm(X_Train)
X_Test = min_max_norm(X_Test)

In [9]:
X_Train.head()

,visit_dow,visit_year,visit_month,holiday_flg,min_visitors,max_visitors,mean_visitors,median_visitors,count_visitors,air_genre_name,air_area_name,latitude,longitude,rv_sum,rv_mean,rvd_mean,date_int,var_max_lat,var_max_long,long_plus_lat,air_store_id_label,weight
0,0.833333,0.0,0.818182,0.0,0.015267,0.021689,0.056300,0.052980,0.588235,0.615385,0.686275,0.225246,0.671814,0.007813,0.041255,0.015682,0.087879,0.774754,0.328186,0.470513,0.466184,0.071467
1,0.166667,0.0,0.818182,0.0,0.000000,0.050228,0.108393,0.079470,0.558824,0.538462,0.254902,0.142939,0.329995,0.007813,0.041255,0.015682,0.087492,0.857061,0.670005,0.238182,0.106280,0.066540
2,0.000000,0.0,0.363636,0.0,0.007634,0.018265,0.047908,0.043046,0.955882,0.846154,0.774510,0.230149,0.670620,0.007813,0.041255,0.015682,0.041566,0.769851,0.329380,0.471988,0.454106,0.003146
3,0.333333,0.0,0.909091,0.0,0.007634,0.073059,0.129653,0.105960,0.955882,0.307692,0.000000,0.034902,0.014012,0.014706,0.116162,0.024232,0.097665,0.965098,0.985988,0.009338,0.992754,0.122323
4,0.333333,0.0,0.545455,0.0,0.045802,0.052511,0.171065,0.158940,0.588235,0.846154,0.941176,0.136532,0.376468,0.007813,0.041255,0.015682,0.058618,0.863468,0.623532,0.262018,0.131643,0.009411


Autoencoder patameters

In [0]:
# Hyperparameters
AE_learning_rate = 0.01
AE_epochs = 500
AE_batch_size = 256

AE_train_flag = True # 학습시 사용될 flag

# Network Parameters
AE_n_hidden_1 = 16 # 1st layer num features
AE_n_hidden_2 = 8 # 2nd layer num features (the latent dim)
AE_n_input = X_Train.shape[0]
AE_input_shape = X_Train.shape[1]

iterations = int(AE_n_input / AE_batch_size)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, AE_input_shape])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([AE_input_shape, AE_n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([AE_n_hidden_1, AE_n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([AE_n_hidden_2, AE_n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([AE_n_hidden_1, AE_input_shape])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([AE_n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([AE_n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([AE_n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([AE_input_shape])),
}

# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    e_layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    e_layer_2 = tf.nn.relu(tf.add(tf.matmul(e_layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return e_layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    d_layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    d_layer_2 = tf.nn.relu(tf.add(tf.matmul(d_layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return d_layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
#AE_loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
AE_msle = tf.keras.losses.MeanSquaredLogarithmicError()
AE_loss = tf.pow(AE_msle(y_true, y_pred), 0.5)
#AE_loss = tf.reduce_mean(tf.pow(tf.log(y_true, name ='log_y') - tf.log(y_pred, name ='log_pred'), 2))
AE_optimizer = tf.train.AdamOptimizer(AE_learning_rate).minimize(AE_loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

AE_test_loss_best = 100
display_step = 25

autoencoder training

In [19]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    if AE_train_flag: 
        # Training
        for i in range(AE_epochs):
            total_loss = 0 
            for j in range(iterations):
                randidx = np.random.randint(AE_n_input, size=AE_batch_size)                
                batch_x = X_Train.loc[randidx, :]
                # Run optimization op (backprop) and cost op (to get loss value)
                _, l = sess.run([AE_optimizer, AE_loss], feed_dict={X: batch_x})
                # Display logs per step
                total_loss += l
            # Testing
            #AE_train_loss = sess.run(AE_loss, feed_dict={X: X_Train})
            AE_train_loss = total_loss / iterations            
            # Testing
            AE_test_loss = sess.run(AE_loss, feed_dict={X: X_Test})
            

            if (i+1) % display_step == 0:
                print(" %d epoch - Autoencoder Training Loss: %.4f" % (i, AE_train_loss))
                print(" %d epoch -  Autoencoder Test Loss: %.4f" % (i, AE_test_loss))

            if AE_train_loss < AE_test_loss_best :
                AE_test_loss_best = AE_train_loss
                X_Train_Encode = sess.run(encoder_op, feed_dict={X: X_Train})
                X_Test_Encode = sess.run(encoder_op, feed_dict={X: X_Test})

 24 epoch - Autoencoder Training Loss: 0.2269
 24 epoch -  Autoencoder Test Loss: 0.2226
 49 epoch - Autoencoder Training Loss: 0.2178
 49 epoch -  Autoencoder Test Loss: 0.2202
 74 epoch - Autoencoder Training Loss: 0.2172
 74 epoch -  Autoencoder Test Loss: 0.2198
 99 epoch - Autoencoder Training Loss: 0.2181
 99 epoch -  Autoencoder Test Loss: 0.2202
 124 epoch - Autoencoder Training Loss: 0.2176
 124 epoch -  Autoencoder Test Loss: 0.2206
 149 epoch - Autoencoder Training Loss: 0.2180
 149 epoch -  Autoencoder Test Loss: 0.2202
 174 epoch - Autoencoder Training Loss: 0.2172
 174 epoch -  Autoencoder Test Loss: 0.2195
 199 epoch - Autoencoder Training Loss: 0.2172
 199 epoch -  Autoencoder Test Loss: 0.2192
 224 epoch - Autoencoder Training Loss: 0.2172
 224 epoch -  Autoencoder Test Loss: 0.2194
 249 epoch - Autoencoder Training Loss: 0.2171
 249 epoch -  Autoencoder Test Loss: 0.2196
 274 epoch - Autoencoder Training Loss: 0.2170
 274 epoch -  Autoencoder Test Loss: 0.2192
 299 ep

DNN 모델링

In [20]:
# Network Parameters
n_hidden_1 = 16 # 1st layer #
n_hidden_2 = 16 # 2nd layer #
#n_input    = X_Train.shape[1] # data input (# of feature)
n_input    = AE_n_hidden_2 # data input (# of feature)
n_classes  = 1 # total classes (1: death, 0: survive)

# tf Graph input
with tf.device('/gpu:0'):
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    lr = tf.placeholder("float", shape=[])


# Create model
def multilayer_perceptron(_X, _weights, _biases):
    with tf.device('/gpu:0'):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
        #layer_2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])), 0.5)        
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return tf.matmul(layer_2, _weights['out']) + _biases['out']

# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!!

with tf.device('/gpu:0'):
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

print("Network Ready to Go!")

Network Ready to Go!


In [21]:
X_Train_Encode.shape

(201686, 8)

In [22]:
tf.set_random_seed(0)
# Parameters
learning_rate_value   = 0.01
training_epochs = 500
batch_size      = 256
display_step    = 25
log_step = int(training_epochs/display_step)
# Add ops to save and restore all the variables.
Model_saver = tf.train.Saver(max_to_keep=None)

# Construct model
with tf.device('/gpu:0'):
    pred = multilayer_perceptron(x, weights, biases)
    # Define loss and optimizer    
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    cost = tf.pow(msle(y, pred), 0.5)
    #cost = tf.reduce_mean(tf.pow(y - pred, 2))
    optm = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)    

# Initializing the variables
init = tf.global_variables_initializer()
n_train = X_Train_Encode.shape[0]

# Train Model
# Launch the graph
sess_run = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
sess_run.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_train/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        randidx = np.random.randint(n_train, size=batch_size)
        batch_xs = X_Train_Encode[randidx, :]
        batch_ys = Y_Train.loc[randidx, :]
        # Fit training using batch data
        if epoch < 10:
            sess_run.run(optm, feed_dict={x: batch_xs, y: batch_ys, lr: learning_rate_value})
        elif epoch < 20:
            sess_run.run(optm, feed_dict={x: batch_xs, y: batch_ys, lr: 0.9*learning_rate_value})
        else:
            sess_run.run(optm, feed_dict={x: batch_xs, y: batch_ys, lr: 0.81*learning_rate_value})
        # Compute average loss
        avg_cost += sess_run.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
        # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_loss = sess_run.run(cost, feed_dict={x: batch_xs, y: batch_ys})
        print(" Training Loss: %.4f" % (train_loss))
        test_loss = sess_run.run(cost, feed_dict={x: X_Test_Encode, y: Y_Test})
        print(" Test Loss: %.4f" % (test_loss))


print("Optimization Finished!")
sess_run.close()
print("Session closed.")

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

Epoch: 024/500 cost: 0.800018800
 Training Loss: 0.8329
 Test Loss: 0.8002
Epoch: 049/500 cost: 0.798177787
 Training Loss: 0.8396
 Test Loss: 0.7976
Epoch: 074/500 cost: 0.800312100
 Training Loss: 0.7690
 Test Loss: 0.7981
Epoch: 099/500 cost: 0.798942703
 Training Loss: 0.7895
 Test Loss: 0.7982
Epoch: 124/500 cost: 0.798201693
 Training Loss: 0.7849
 Test Loss: 0.7979
Epoch: 149/500 cost: 0.795620020
 Training Loss: 0.8352
 Test Loss: 0.7970
Epoch: 174/500 cost: 0.797775379
 Training Loss: 0.8029
 Test Loss: 0.7970
Epoch: 199/500 cost: 0.797584459
 Training Loss: 0.8225
 Test Loss: 0.7969
Epoch: 224/500 cost: 0.796914688
 Training Loss: 0.7648
 Test Loss: 0.7981
Epoch: 249/500 cost: 